In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

play_data = pd.read_csv('./data/plays.csv')
pff_data = pd.read_csv('./data/PFFScoutingData.csv')

discard_results = ['Non-Special Teams Result']#,'Blocked Punt']
punt_plays = play_data[(play_data['specialTeamsPlayType']=='Punt')&(~play_data['specialTeamsResult'].isin(discard_results))]
pff_punt = pff_data.merge(punt_plays[['gameId','playId']],on=['gameId','playId'],how='inner')

fb_land = pd.read_csv('./ReducedData/fb_land.csv')

# 2018 Football Tracking Data
tracking_game = pd.read_csv('./data/tracking2018.csv', chunksize=10**5)
football_tracking = pd.concat((x.query("displayName == 'football'") for x in tracking_game), ignore_index=True)
punt_fbtrack_18 = football_tracking.merge(punt_plays[['gameId','playId']],on=['gameId','playId'])
punt_fbtrack_18['time'] = pd.to_datetime(punt_fbtrack_18['time'])

# 2019 Football Tracking Data
tracking_game = pd.read_csv('./data/tracking2019.csv', chunksize=10**5)
football_tracking = pd.concat((x.query("displayName == 'football'") for x in tracking_game), ignore_index=True)
punt_fbtrack_19 = football_tracking.merge(punt_plays[['gameId','playId']],on=['gameId','playId'])
punt_fbtrack_19['time'] = pd.to_datetime(punt_fbtrack_19['time'])

# 2020 Football Tracking Data
tracking_game = pd.read_csv('./data/tracking2020.csv', chunksize=10**5)
football_tracking = pd.concat((x.query("displayName == 'football'") for x in tracking_game), ignore_index=True)
punt_fbtrack_20 = football_tracking.merge(punt_plays[['gameId','playId']],on=['gameId','playId'])
punt_fbtrack_20['time'] = pd.to_datetime(punt_fbtrack_20['time'])

# Combine all above tracking data
punt_fbtrack = pd.concat([punt_fbtrack_18,punt_fbtrack_19,punt_fbtrack_20]) \
                .sort_values(by=['gameId','playId','frameId']) \
                .drop(columns=['o','dir','nflId','displayName','jerseyNumber','position','team']) \
                .reset_index(drop=True)

### Snap Frame

When the play has a 'ball_snap' event, always use it.

When the 1st frame has a value of dx > 0.1, look at frame 1

For the 4 games remaining, find first frame with a somewhat large dx.

In [2]:
fb_use = punt_fbtrack.merge(fb_land[['gameId','playId']]).copy()
fb_use['dx'] = fb_use.groupby(['gameId','playId'])['x'].diff(-1)
snaps = fb_use[fb_use['event']=='ball_snap']

no_snap = fb_use[~(fb_use['gameId'].astype(str) + fb_use['playId'].astype(str)) \
                 .isin((snaps['gameId'].astype(str) + snaps['playId'].astype(str)).values)]

frame1_snap = no_snap[(no_snap['frameId']==1)&(np.abs(no_snap['dx'])>0.1)]

remaining = fb_use.merge(no_snap[(no_snap['frameId']==1)&(np.abs(no_snap['dx'])<=0.1)][['gameId','playId']])
early_snap = remaining[np.abs(remaining['dx'])>0.1].groupby(['gameId','playId']).head(1)

In [3]:
snap_frames = pd.concat([snaps,frame1_snap,early_snap])[['gameId','playId','frameId']]
snap_frames.rename({'frameId':'snap_frame'},axis=1,inplace=True)

In [83]:
snap_frames.sort_values(['gameId','playId']).to_csv('./ReducedData/snapframe.csv',index=False)